# REINFORCEMENT LEARNING 
in this notebook we use Open-AI gym to implement a RL problem

## Imports

***mount drive***

In [1]:
from google.colab import drive 
drive.mount("/content/drive")

Mounted at /content/drive


***general imports***

In [2]:
import numpy as np 
import random 
import gym
from collections import deque
import os 

***tensorflow imports***

In [3]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.optimizers import Adam

***making a dummy video player driver***

In [4]:
os.environ["SDL_VIDEODRIVER"] = "dummy"

***installing pygame***

In [5]:
!pip install gym[classic_control]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 66.8 MB/s eta 0:00:00


## RL Environment 

***initializing the cartpole environment***

In [6]:
env = gym.make("CartPole-v1", new_step_api=True)

states = env.observation_space.shape[0]
actions = env.action_space.n
batch_size = 32
n_episodes = 200

print("this is number of actions:", actions)
print("this is number of states:", states)

this is number of actions: 2
this is number of states: 4


***saving the model***

In [7]:
output_dir = "/content/drive/MyDrive/Youtube/9 - RL/model/"

if not os.path.exists(output_dir):
  os.makedirs(output_dir)

## Agent

***defining the DQG-Agent Class***

In [8]:
class DQNAgent:

    def __init__(self, states, actions):
        self.states = states
        self.actions = actions
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95 
        self.epsilon = 1.0
        self.epsilon_decay = 0.995 
        self.epsilon_min = 0.01 
        self.learning_rate = 0.001
        self.model = self._build_model()
    
    def _build_model(self):
        model = Sequential()
        model.add(Dense(32, activation='relu', input_dim=self.states))
        model.add(Dense(32, activation='relu')) 
        model.add(Dense(self.actions, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate)) 
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, 
                            reward, next_state, done))
    
    def train(self, batch_size): 
        minibatch = random.sample(self.memory, batch_size) 
        for state, action, reward, next_state, done in minibatch: 
            target = reward 
            if not done: 
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0])) 
            target_f = self.model.predict(state) 
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)  
        if self.epsilon > self.epsilon_min: 
            self.epsilon *= self.epsilon_decay 

    def act(self, state):
        if np.random.rand() <= self.epsilon: 
            return random.randrange(self.actions) 
        act_values = self.model.predict(state) 
        return np.argmax(act_values[0]) 
        
    def save(self, name):
        self.model.save_weights(name)

    def load(self, name):
        self.model.load_weights(name)

***instantiating the DQN-Agent***

In [9]:
agent = DQNAgent(states, actions) 

## Train the DQN-Agent

In [10]:
for e in range(n_episodes):
    
    state = env.reset()
    state = np.reshape(state, [1, states]) 
    
    done = False
    time = 0 
    while not done: 
        action = agent.act(state) 
        next_state, reward, done, _, _ = env.step(action) 
        reward = reward if not done else -10 
        next_state = np.reshape(next_state, [1, states]) 
        agent.remember(state, action, reward, next_state, done)       
        state = next_state        
        if done:  
            print( "episode: {}/{}, score: {}, e: {:.2}"
                  .format(e, n_episodes-1, time, agent.epsilon))
        time += 1
        
        env.render()
        
    if len(agent.memory) > batch_size:
        agent.train(batch_size) 
    
    if e % 50 == 0:
        agent.save(output_dir + "weights_" + '{:04d}'.format(e) + ".hdf5")

/usr/local/lib/python3.9/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 19ms/step
episode: 150/199, score: 67, e: 0.47
1/1 [==============================] - 0s 41ms/step
episode: 151/199, score: 30, e: 0.47
1/1 [==============================] - 0s 28ms/step
episode: 152/199, score: 47, e: 0.47
1/1 [==============================] - 0s 24ms/step
episode: 153/199, score: 82, e: 0.47
1/1 [==============================] - 0s 33ms/step
episode: 154/199, score: 69, e: 0.46
1/1 [==============================] - 0s 40ms/step
episode: 155/199, score: 58, e: 0.46
1/1 [==============================] - 0s 24ms/step
episode: 156/199, score: 79, e: 0.46
1/1 [==============================] - 0s 27ms/step
episode: 157/199, score: 45, e: 0.46
1/1 [==============================] - 0s 46ms/step
episode: 158/199, score: 17, e: 0.46
1/1 [==============================] - 0s 22ms/step
episode: 159/199, score: 39, e: 0.45
1/1 [==============================] - 0s 21ms/step
episode